In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import glob
import torch
from PIL import Image
import re

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,8)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 18}
plt.rc('font', **font)

In [ ]:
%matplotlib inline

In [ ]:
transform = transforms.Compose([
    transforms.Resize((576, 704)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def parse_img_num(img):
    m = re.search(r"frame(\d+).png", img)
    if m:
        return f"{int(m.groups()[0]):06d}"
    print(img)
    raise Exception("Couldn't parse number")

In [ ]:
class MySampler(torch.utils.data.Sampler):
    def __init__(self, end_idx, seq_length):
        indices = []
        for i in range(len(end_idx) - 1):
            start = end_idx[i]
            end = end_idx[i + 1] - seq_length
            if start > end:
                indices.append(torch.arange(end, start))
            else:
                indices.append(torch.arange(start, end))
        indices = torch.cat(indices)
        self.indices = indices
        
    def __iter__(self):
        indices = self.indices[torch.randperm(len(self.indices))]
        return iter(indices.tolist())
    
    def __len__(self):
        return len(self.indices)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_paths, seq_length, transform, length): #csv_file, 
        self.image_paths = image_paths
        self.seq_length = seq_length
        self.transform = transform
        self.length = length
        
    def __getitem__(self, index):
        start = index
        end = index + self.seq_length
        print('Getting images from {} to {}'.format(start, end))
        indices = list(range(start, end))
        images = []
        for i in indices:
            image_path = self.image_paths[i][0]
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)
            images.append(image)
        x = torch.stack(images)
        y = torch.tensor([self.image_paths[start][1]], dtype=torch.long)
        
        return x, y
    
    def __len__(self):
        return self.length

In [ ]:
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
class_paths = [d.path for d in os.scandir(root_dir) if d.is_dir]

In [ ]:
class_paths

In [ ]:
one_hot_classes = [[1,0,1,0,0],[1,0,0,0,1],[1,0,0,0,0],[1,0,0,1,0],[0,1,0,0,0]]

In [ ]:
df = pd.read_csv('../train-valid-splits-video-scratch/valid.csv')
df.head()

In [ ]:
class_names = ['exp_and','exp_fs','exp','exp_fj','bur']
one_hot_classes = [[1,0,1,0,0],[1,0,0,0,1],[1,0,0,0,0],[1,0,0,1,0],[0,1,0,0,0]]
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
     for d in os.scandir(class_path):
        if d.is_dir:
            if d.path in df.videos.values:
                paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
                # Add class idx to paths
                paths = [(p, one_hot_classes[c]) for p in paths]
                class_image_paths.extend(paths)
                end_idx.extend([len(paths)])

In [ ]:
end_idx[:10]

In [ ]:
len(paths)

In [ ]:
len(class_image_paths)

In [ ]:
len(end_idx)

In [ ]:
sum(end_idx)

In [ ]:
end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
end_idx[:10]

In [ ]:
# class_image_paths = []
# end_idx = []
# for c, class_path in enumerate(class_paths):
#     for d in os.scandir(class_path):
#         if d.is_dir:
#             paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
#             # Add class idx to paths
#             paths = [(p, one_hot_classes[c]) for p in paths]
#             class_image_paths.extend(paths)
#             end_idx.extend([len(paths)])
            
# end_idx = [0, *end_idx]
# end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
    for d in os.scandir(class_path):
        if d.is_dir:
            if d.path in df.videos.values:
                paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
                # Add class idx to paths
                paths = [(p, one_hot_classes[c]) for p in paths]
                if len(paths)>=16:
                    class_image_paths.extend(paths)
                    end_idx.extend([len(paths)])
                
end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
end_idx

In [ ]:
seq_length = 16

In [ ]:
indices = []
for i in range(len(end_idx) - 1):
    start = end_idx[i]
    print(start)
    end = end_idx[i + 1] - seq_length
    print(end)
    #if start > end:
    indices.append(torch.arange(start, end))
    print(indices)
#             else:
#                 indices.append(torch.arange(start, end))
indices = torch.cat(indices)

In [ ]:
indices[10:20]

In [ ]:
indices = indices[torch.randperm(len(indices))]
l = iter(indices.tolist())
    

In [ ]:
indices

In [ ]:
next(iter(l))

In [ ]:
class MySampler_test(torch.utils.data.Sampler):
    def __init__(self, end_idx, seq_length):
        indices = []
        for i in range(len(end_idx) - 1):
            start = end_idx[i]
            print(start)
            end = end_idx[i + 1] - seq_length
            print(end)
#             if start > end:
#                 indices.append(torch.arange(end, start))
#             else:
            indices.append(torch.arange(start, end))
        indices = torch.cat(indices)
        self.indices = indices
        
    def __iter__(self):
        indices = self.indices[torch.randperm(len(self.indices))]
        return iter(indices.tolist())
    
    def __len__(self):
        return len(self.indices) 

In [ ]:
sampler.indices

In [ ]:
sampler = MySampler_test(end_idx, seq_length)

In [ ]:
sampler = MySampler(end_idx, seq_length)

In [ ]:
len(sampler)

In [ ]:
dataset = MyDataset(
    image_paths=class_image_paths,
    seq_length=seq_length,
    transform=transform,
    length=len(sampler))


In [ ]:
loader = DataLoader(
    dataset,
    batch_size=1,
    sampler=sampler,
    drop_last=True,
    num_workers=0
)

In [ ]:
# for data, target in loader:
#     print(data.shape)
#     print(target.shape)

In [ ]:
t = next(iter(loader))

In [ ]:
import torchvision

In [ ]:
class_names = ['exp_and','exp_fs','exp','exp_fj','bur']

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(30,30))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader))
inputs = inputs.squeeze(dim = 0)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)


for i, f in enumerate(one_hot_classes):
    if np.array_equal(classes[0][0].numpy(), np.asarray(f)):
        title = class_names[i]
        
        
imshow(out, title=title)

In [ ]:
classes

In [ ]:
inputs